# ws 04

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout #Conv2D,  MaxPool2D 

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
target_img_shape=(128, 128) 

train_dir = './train_set'  

val_dir = './val_set'  

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size=target_img_shape,
                                                 batch_size=32, 
                                                 class_mode='sparse') 

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) 
val_set = val_datagen.flow_from_directory(val_dir,
                                            target_size=target_img_shape,
                                            batch_size=32,
                                            class_mode='sparse')

In [ ]:
print(train_set.class_indices)
for image_batch, labels_batch in train_set:
    print(image_batch.shape)
    print(labels_batch.shape)
    
    img = image_batch[0] - image_batch[0].min()
    img /= 275.0
    
    print('Min Max:', img.min(), img.max())
    plt.imshow(img)

    print('class:', labels_batch[0])
    
    break

In [ ]:
ids, counts = np.unique(train_set.classes, return_counts=True)

print(ids)
print(counts)

In [ ]:
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
labels   

for i in ids:
    print('{:>8} = {}' . format(labels[i], counts[i]))

In [ ]:
label_names = [k for k in train_set.class_indices]
label_names

In [ ]:
import pandas as pd

df_train_labels = pd.DataFrame({'Label':label_names, 'Count':counts})
df_train_labels.set_index('Label', inplace=True)
df_train_labels

In [ ]:
df_train_labels.plot.bar()
plt.show()

In [ ]:
ids, counts = np.unique(val_set.classes, return_counts=True)

print(ids)
print(counts)

In [ ]:
_, train_count = np.unique(train_set.classes, return_counts=True)
_, val_count = np.unique(val_set.classes, return_counts=True)

print('Ratio Validation/Training set:', val_count/train_count * 100)

In [ ]:
print(train_set[0][0][0].min(),train_set[0][0][1].max())

plt.imshow(train_set[0][0][0])
plt.show()

In [ ]:
in_shape = (target_img_shape[0], target_img_shape[1], 3)  # in_shape = (64,64,3)
in_shape

In [ ]:
base_model = ResNet50(include_top=False, weights='imagenet',
                   input_shape=in_shape)  

model = Sequential()

model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))  
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu')) 
model.add(Dropout(0.4))
model.add(Dense(3, activation='softmax'))  

model.summary()

In [ ]:
base_model.trainable = False 

In [ ]:
print("Trainable..\n---")
for variable in model.trainable_variables:
    print(variable.name)  

In [ ]:
model.summary()

In [ ]:
import time

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', 
             metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', verbose=1, patience=5) # 
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True)

start = time.time()
history = model.fit(train_set, 
                     validation_data=val_set, 
                    epochs=20, verbose=1, callbacks=[es, mc])

end = time.time()
print("Time Taken: {:.2f} minutes".format((end - start)/60))

In [ ]:
plt.figure(figsize=(10, 3.5))
plt.subplot(1, 2, 1)

plt.title('Loss')
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'],'r', lw=3.2, label='Validation')
plt.legend()

plt.subplot(1, 2, 2)
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='Training')
plt.plot(history.history['val_accuracy'], 'r', lw=3.2, label='Validation')

plt.legend()
plt.show()

In [ ]:
print(len(val_set))
acc = model.evaluate(val_set, verbose=1)
print('score = {:.3f}' .format(acc[1]))

## Predict

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def predict(img_fname):
    img = load_img(img_fname, target_size=target_img_shape) 
    plt.imshow(img)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    pred = model.predict(img)  
    pred_cls = labels[np.argmax(pred, -1)[0]]   
    print('Prediction:', pred_cls, pred[0].round(3))


In [ ]:
predict('dog010-2.jpg')

In [ ]:
predict('./elephant.jpeg')

In [ ]:
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
labels  

In [ ]:
labels[np.argmax(pred, -1)[0]]

# ws 05

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions

In [ ]:
target_img_shape=(128, 128)  

train_dir = './train_set'  

val_dir = './val_set' 

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) 


train_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size=target_img_shape,
                                                 batch_size=32,
                                                 class_mode='sparse')

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) 
    
val_set = val_datagen.flow_from_directory(val_dir,
                                            target_size=target_img_shape,
                                            batch_size=32,
                                            class_mode='sparse')

In [ ]:
print(train_set.class_indices)


for image_batch, labels_batch in train_set:
    print(image_batch.shape)
    print(labels_batch.shape)
    
    img = image_batch[0] - image_batch[0].min()
    img /= 2.0
   
    print(img.min(), img.max())
    plt.imshow(img)

    print('class:', labels_batch[0])    
    break

In [ ]:
in_shape = (target_img_shape[0], target_img_shape[1], 3)  


base_model = MobileNetV2(include_top=False, weights='imagenet',
                   input_shape=in_shape)  

model = Sequential()

model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))  
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))  
model.add(Dropout(0.4))
model.add(Dense(3, activation='softmax'))  

model.summary()